https://github.com/dbabichenko/python_for_data_and_analytics/tree/master/12%20-%20Classification

In [1]:
# Required Python Machine learning Packages
import pandas as pd
import numpy as np
import operator

# To split the dataset into train and test datasets
from sklearn.model_selection import train_test_split

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier

# To calculate the accuracy score of the model
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def label_encode(df, columns):
    for col in columns:
        le = LabelEncoder()
        col_values_unique = list(df[col].unique())
        le_fitted = le.fit(col_values_unique)
 
        col_values = list(df[col].values)
        #le.classes_
        col_values_transformed = le.transform(col_values)
        df[col] = col_values_transformed

In [3]:
def classify(predictors, response, classifier = 'svm', kern='rbf', neighbors=3, kfolds=0, report=False, features_select=0):        
    # split X and y into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(predictors, response, test_size=0.20, random_state=1)

    
    if classifier == 'nb':
        cl = GaussianNB() # instantiate model
        msg = 'Naive Bayes'
    elif classifier == 'svm':
        cl = svm.SVC(kernel=kern) # instantiate model
        msg = 'SVM with ' + kern + ' kernel'
    elif classifier == 'knn':
        cl = KNeighborsRegressor(n_neighbors=neighbors)
        msg = 'KNN with k=' + str(neighbors)
    elif classifier == 'rf':
        # Instantiate model with  decision trees
        # Note that the number of decision trees is denoted
        # using the neighbors parameter, which is set to 3 by default
        cl = RandomForestRegressor(n_estimators = 1000, random_state = 42)
        msg = 'Random Forest with ' + str(neighbors) + ' decision trees'
    elif classifier == 'dtree':
        cl = DecisionTreeClassifier(min_samples_split=20, random_state=99)
        msg = 'Decision tree'
    elif classifier == 'logisticregression':
        cl = LogisticRegression()
        msg = 'Logistic Regression'
        
    model = cl.fit(X_train, y_train)
    y_pred_class = model.predict(X_test)
    print(msg + ' model accuracy score: ', metrics.accuracy_score(y_test, y_pred_class.round()))
    
    if kfolds > 0:
        # Perform k-fold cross validation
        scores = cross_val_score(model, predictors, response, cv=kfolds)
        print('Cross-validated score:', scores.mean())
    
    if report == True:
        print('Confusion matrix')
        print(metrics.confusion_matrix(y_test, y_pred_class)) # confusion matrix
        print('Classification report')
        target_names=['low','med','high']
        print(metrics.classification_report(y_test, y_pred_class, target_names=target_names))
        
        print('______________________________')
    
    if features_select > 0:
        test = SelectKBest(score_func=chi2, k=features_select)
        fit = test.fit(predictors, response)
        
        # summarize scores
        np.set_printoptions(precision=3)
        # print(fit.scores_)
        features_scores = {}
        for i in range(0, len(predictors.columns)):
            features_scores[predictors.columns[i]] = fit.scores_[i]
            
        # print(features_scores)
        sorted_features_scores = sorted(features_scores.items(), key=operator.itemgetter(1))
        print(sorted_features_scores[-5:])        

In [4]:
def decompose(predictors):
    # feature extraction
    pca = PCA(n_components=3)
    fit = pca.fit(predictors)
    
    # summarize components
    # print(fit.explained_variance_ratio_)
    # print(fit.components_)
    return pd.DataFrame(fit.components_)

In [5]:
df = pd.read_csv("311TrimmedGrouped.csv")
df.head()

,Unnamed: 0,REQUEST_ID,CREATED_ON,REQUEST_TYPE,REQUEST_ORIGIN,STATUS,DEPARTMENT,NEIGHBORHOOD,COUNCIL_DISTRICT,WARD,...,daily.temperatureMinTime,daily.time,daily.uvIndex,daily.uvIndexTime,daily.visibility,daily.windBearing,daily.windGust,daily.windGustTime,daily.windSpeed,REQUEST_GROUP
0,13,205276,2017-12-31T13:42:00,Potholes,Website,1,DPW - Street Maintenance,Beechview,4.0,19.0,...,1514721600,1514696400,1,1514732400,9.10,261,5.37,1514696400,0.56,Roads
1,75,205180,2017-12-29T17:45:00,Potholes,Website,1,DPW - Street Maintenance,Manchester,6.0,21.0,...,1514541600,1514523600,1,1514559600,8.58,253,8.96,1514577600,1.31,Roads
2,96,205151,2017-12-29T14:58:00,Potholes,Call Center,1,DPW - Street Maintenance,Regent Square,9.0,14.0,...,1514541600,1514523600,1,1514559600,8.28,252,8.98,1514577600,1.26,Roads
3,112,205130,2017-12-29T14:03:00,Potholes,Call Center,1,DPW - Street Maintenance,Brookline,4.0,19.0,...,1514541600,1514523600,1,1514559600,8.59,253,9.22,1514577600,1.30,Roads
4,133,205098,2017-12-29T12:40:00,Potholes,Call Center,1,DPW - Street Maintenance,Lincoln Place,5.0,31.0,...,1514541600,1514523600,1,1514559600,8.32,252,9.37,1514577600,1.29,Roads


In [6]:
df.fillna(0, inplace=True) #fill zero instances of weather
df = df.reset_index()

In [7]:
df.columns

Index(['index', 'Unnamed: 0', 'REQUEST_ID', 'CREATED_ON', 'REQUEST_TYPE',
       'REQUEST_ORIGIN', 'STATUS', 'DEPARTMENT', 'NEIGHBORHOOD',
       'COUNCIL_DISTRICT', 'WARD', 'TRACT', 'PUBLIC_WORKS_DIVISION',
       'PLI_DIVISION', 'POLICE_ZONE', 'FIRE_ZONE', 'X', 'Y', 'GEO_ACCURACY',
       'current.apparentTemperature', 'current.cloudCover', 'current.dewPoint',
       'current.humidity', 'current.icon', 'current.precipIntensity',
       'current.precipProbability', 'current.precipType', 'current.pressure',
       'current.summary', 'current.temperature', 'current.time',
       'current.uvIndex', 'current.visibility', 'current.windBearing',
       'current.windGust', 'current.windSpeed',
       'daily.apparentTemperatureHigh', 'daily.apparentTemperatureHighTime',
       'daily.apparentTemperatureLow', 'daily.apparentTemperatureLowTime',
       'daily.apparentTemperatureMax', 'daily.apparentTemperatureMaxTime',
       'daily.apparentTemperatureMin', 'daily.apparentTemperatureMinTime',
 

In [8]:
df.corr()

,index,Unnamed: 0,REQUEST_ID,STATUS,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,...,daily.temperatureMin,daily.temperatureMinTime,daily.time,daily.uvIndex,daily.uvIndexTime,daily.visibility,daily.windBearing,daily.windGust,daily.windGustTime,daily.windSpeed
index,1.000000,-0.051815,0.052628,-0.049882,-0.071076,0.009420,-0.009921,-0.028873,0.009420,-0.075824,...,-0.109016,0.053698,0.053694,-0.125977,0.053661,-0.071600,0.024039,-0.043074,0.053474,-0.015342
Unnamed: 0,-0.051815,1.000000,-0.999832,-0.009968,0.041724,-0.029995,-0.012563,-0.001909,-0.029995,0.015102,...,-0.246287,-0.996791,-0.996810,-0.032840,-0.996819,-0.030910,0.075709,0.401221,-0.996832,0.415942
REQUEST_ID,0.052628,-0.999832,1.000000,0.010486,-0.041505,0.029464,0.012534,0.001780,0.029464,-0.015286,...,0.232976,0.997759,0.997771,0.020121,0.997777,0.029375,-0.075120,-0.395048,0.997785,-0.407309
STATUS,-0.049882,-0.009968,0.010486,1.000000,0.005129,0.004965,0.008233,-0.025449,0.004965,-0.022022,...,-0.019003,0.012299,0.012262,-0.014554,0.012261,-0.006097,0.006358,0.009309,0.012293,0.007724
COUNCIL_DISTRICT,-0.071076,0.041724,-0.041505,0.005129,1.000000,-0.610653,-0.383170,-0.252472,-0.610653,0.332147,...,-0.014441,-0.042063,-0.042095,-0.018534,-0.042107,-0.017932,0.015295,0.022818,-0.042138,0.025142
WARD,0.009420,-0.029995,0.029464,0.004965,-0.610653,1.000000,0.444382,0.072012,1.000000,-0.138735,...,0.023906,0.029914,0.029998,0.055869,0.030016,0.026315,-0.017567,-0.018364,0.030069,-0.032200
TRACT,-0.009921,-0.012563,0.012534,0.008233,-0.383170,0.444382,1.000000,0.113109,0.444382,0.009178,...,0.004035,0.013886,0.013969,0.022993,0.013980,0.018206,-0.018241,-0.007101,0.014017,-0.009092
PUBLIC_WORKS_DIVISION,-0.028873,-0.001909,0.001780,-0.025449,-0.252472,0.072012,0.113109,1.000000,0.072012,0.378562,...,0.006943,0.002764,0.002765,0.023192,0.002776,0.006160,0.004018,-0.001536,0.002802,-0.002251
PLI_DIVISION,0.009420,-0.029995,0.029464,0.004965,-0.610653,1.000000,0.444382,0.072012,1.000000,-0.138735,...,0.023906,0.029914,0.029998,0.055869,0.030016,0.026315,-0.017567,-0.018364,0.030069,-0.032200
POLICE_ZONE,-0.075824,0.015102,-0.015286,-0.022022,0.332147,-0.138735,0.009178,0.378562,-0.138735,1.000000,...,0.004851,-0.015536,-0.015512,0.024104,-0.015507,-0.000418,0.004428,0.008654,-0.015506,0.005043


In [9]:
# define X and y
X = df[['current.cloudCover', 'current.humidity',
       'current.icon', 'current.precipIntensity', 'current.precipProbability',
       'current.precipType', 'current.summary',
       'current.temperature', 'current.uvIndex',
       'current.visibility', 'current.windBearing', 'current.windGust',
       'current.windSpeed', 'daily.cloudCover', 'daily.humidity', 'daily.icon', 'daily.moonPhase',
       'daily.precipAccumulation', 'daily.precipIntensity', 'daily.precipIntensityMax', 
       'daily.precipProbability', 'daily.precipType', 'daily.pressure',
       'daily.summary', 'daily.temperatureHigh',  
     'daily.uvIndex', 'daily.visibility',
       'daily.windBearing', 'daily.windGust', 'daily.windSpeed']]
label_encode(X, X.columns.values)

# y = df[['NEIGHBORHOOD']]
# y = df['DEPARTMENT']
y = df[['REQUEST_GROUP']]  

label_encode(y, y.columns.values)
# y = y['REQUEST_GROUP']

C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [10]:
X.corr()

,current.cloudCover,current.humidity,current.icon,current.precipIntensity,current.precipProbability,current.precipType,current.summary,current.temperature,current.uvIndex,current.visibility,...,daily.precipProbability,daily.precipType,daily.pressure,daily.summary,daily.temperatureHigh,daily.uvIndex,daily.visibility,daily.windBearing,daily.windGust,daily.windSpeed
current.cloudCover,1.000000,0.219752,0.612156,0.284193,0.433131,0.398788,0.687319,-0.172083,-0.143001,-0.230114,...,0.388356,0.396291,-0.290961,0.126212,-0.369651,-0.472735,-0.323801,0.075511,0.213596,0.278524
current.humidity,0.219752,1.000000,0.279921,0.244166,0.331615,0.212861,0.066784,-0.002270,-0.521086,-0.407161,...,0.244955,0.150114,-0.239394,0.125117,0.164234,-0.024223,-0.292153,-0.039152,-0.179113,-0.189341
current.icon,0.612156,0.279921,1.000000,0.417408,0.583065,0.415255,0.516385,-0.005653,-0.098155,-0.250693,...,0.385038,0.327423,-0.272930,0.200444,-0.119255,-0.230791,-0.245126,0.038444,0.120122,0.129333
current.precipIntensity,0.284193,0.244166,0.417408,1.000000,0.773073,0.284258,0.096188,0.061160,-0.035435,-0.224758,...,0.371384,0.154788,-0.286921,0.217814,0.000736,-0.087355,-0.174144,-0.088439,0.081215,0.032921
current.precipProbability,0.433131,0.331615,0.583065,0.773073,1.000000,0.549441,0.014156,0.005860,-0.064205,-0.287504,...,0.536900,0.296595,-0.340543,0.191034,-0.094829,-0.210769,-0.261583,-0.053398,0.163863,0.147823
current.precipType,0.398788,0.212861,0.415255,0.284258,0.549441,1.000000,0.086435,-0.146615,-0.061657,-0.269973,...,0.409540,0.505746,-0.230744,0.074169,-0.260090,-0.260736,-0.284757,0.071662,0.274542,0.296868
current.summary,0.687319,0.066784,0.516385,0.096188,0.014156,0.086435,1.000000,-0.062876,-0.059931,-0.037602,...,0.146817,0.216664,-0.163231,0.137421,-0.198564,-0.263003,-0.126983,0.072331,0.097398,0.126039
current.temperature,-0.172083,-0.002270,-0.005653,0.061160,0.005860,-0.146615,-0.062876,1.000000,0.483302,0.063889,...,0.158115,-0.149595,-0.251469,0.179496,0.891066,0.605580,0.098967,-0.118181,-0.290879,-0.433209
current.uvIndex,-0.143001,-0.521086,-0.098155,-0.035435,-0.064205,-0.061657,-0.059931,0.483302,1.000000,0.226315,...,-0.025573,-0.073042,-0.044001,0.027921,0.252126,0.325905,0.094549,-0.028063,-0.086694,-0.140981
current.visibility,-0.230114,-0.407161,-0.250693,-0.224758,-0.287504,-0.269973,-0.037602,0.063889,0.226315,1.000000,...,-0.080385,-0.177338,0.113314,0.028680,0.033977,0.127173,0.620594,-0.038133,0.098157,0.076393


In [13]:
#classify(X, y, classifier='nb', kfolds=10, report=False, features_select=3)
# classify(X, y, classifier='svm', kern='rbf', kfolds=10, report=False, features_select=3)
#X_pca = decompose(X)
#classify(X_pca, y, classifier='svm', kern='rbf', kfolds=10, report=False, features_select=3)
#print(X_pca.shape)
# classify(X, y, classifier='svm', kern='linear', kfolds=10, report=True)
# classify(X, y, classifier='svm', kern='poly', kfolds=10, report=True)
# classify(X, y, classifier='knn', neighbors=3, kfolds=10)
# classify(X, y, classifier='knn', neighbors=5, kfolds=10)
# classify(X, y, classifier='knn', neighbors=7)
# classify(X, y, classifier='rf', neighbors=1000, kfolds=10)
# classify(X, y, classifier='rf', neighbors=2000)
# classify(X, y, classifier='dtree', kfolds=10)
classify(X, y, classifier='logisticregression', kfolds=10, report=True, features_select=5)

C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic Regression model accuracy score:  0.393658471046621


C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A 

Cross-validated score: 0.34792896938601936
Confusion matrix
[[   0    0    0   33  314   18    0]
 [   0    0    0   65  759   56    0]
 [   0    0    0   23  119    7    0]
 [   0    0    0   95 1435  114    0]
 [   0    0    0  147 2482  141    0]
 [   0    0    0   70  774  353    0]
 [   0    0    0   23  409    6    0]]
Classification report
             precision    recall  f1-score   support

        low       0.00      0.00      0.00       365
        med       0.00      0.00      0.00       880
       high       0.00      0.00      0.00       149

avg / total       0.27      0.39      0.28      7443

______________________________
[('daily.precipIntensityMax', 160104.58115881545), ('current.windGust', 222673.68017020245), ('daily.precipAccumulation', 362324.48368737136), ('current.temperature', 1206519.9106679899), ('daily.temperatureHigh', 1616820.509686216)]


C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 3
  .format(len(labels), len(target_names))
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
classify(X, y, classifier='nb', kfolds=10, report=True, features_select=5)

C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A 

Naive Bayes model accuracy score:  0.2883246002955797


C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A 

Cross-validated score: 0.24838012090811165
Confusion matrix
[[  27    3    0   42  111   18  164]
 [  49    3    0  113  304  122  289]
 [  10    0    0   19   45   12   63]
 [  98    1    0  189  511  225  620]
 [ 122    7    0  271 1208  307  855]
 [  38    1    0   93  272  462  331]
 [  26    3    0   36  107    9  257]]
Classification report
             precision    recall  f1-score   support

        low       0.07      0.07      0.07       365
        med       0.17      0.00      0.01       880
       high       0.00      0.00      0.00       149

avg / total       0.32      0.29      0.28      7443

______________________________
[('daily.precipIntensityMax', 160104.58115881545), ('current.windGust', 222673.68017020245), ('daily.precipAccumulation', 362324.48368737136), ('current.temperature', 1206519.9106679899), ('daily.temperatureHigh', 1616820.509686216)]


C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 3
  .format(len(labels), len(target_names))
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
classify(X, y, classifier='dtree', kfolds=10, report=True, features_select=5)

Decision tree model accuracy score:  0.3837162434502217
Cross-validated score: 0.21509366425934867
Confusion matrix
[[  16   39    3   89  167   32   19]
 [  16  143    3  206  356  120   36]
 [   5   22    7   32   57   20    6]
 [  42  155   11  647  533  187   69]
 [  56  288   21  520 1526  259  100]
 [  29  100    6  207  342  478   35]
 [  19   46    1  101  177   55   39]]
Classification report
             precision    recall  f1-score   support

        low       0.09      0.04      0.06       365
        med       0.18      0.16      0.17       880
       high       0.13      0.05      0.07       149

avg / total       0.36      0.38      0.37      7443

______________________________
[('daily.precipIntensityMax', 160104.58115881545), ('current.windGust', 222673.68017020245), ('daily.precipAccumulation', 362324.48368737136), ('current.temperature', 1206519.9106679899), ('daily.temperatureHigh', 1616820.509686216)]


C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 3
  .format(len(labels), len(target_names))


In [16]:
#classify(X, y, classifier='rf', kfolds=10, report=True, features_select=5)

In [19]:
classify(X, y, classifier='rf', neighbors=1000, kfolds=5, report=True)

C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Random Forest with 1000 decision trees model accuracy score:  0.38479107886604863


C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\boblutz\AppData\Local\Continuum\anaconda3\l

Cross-validated score: -1.7465609382341178
Confusion matrix


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [ ]:
classify(X, y, classifier='knn', neighbors=7, kfolds=10, report=True, features_select=5)

In [ ]:
classify(X, y, classifier='knn', neighbors=5, kfolds=10, report=True, features_select=5)

In [ ]:
classify(X, y, classifier='knn', neighbors=3, kfolds=10, report=True, features_select=5)

In [ ]:
# classify(X, y, classifier='svm', kern='poly', kfolds=10, report=True)
# classify(X, y, classifier='svm', kern='linear', kfolds=10, report=True)

In [ ]:
classify(X, y, classifier='svm', kern='rbf', kfolds=10, report=True, features_select=5)

In [ ]:
classify(X, y, classifier='svm', kern='ply', kfolds=10, report=True, features_select=5)

In [ ]:
X_pca = decompose(X)
classify(X_pca, y, classifier='svm', kern='rbf',kfolds=10, report=True, features_select=5)
print(X_pca.shape)